<a href="https://colab.research.google.com/github/HSV-AI/presentations/blob/master/2019/190821_Large_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huntsville AI

## Agenda

* Intro & Welcome
* Project Updates
** T-Shirts
* Deep Learning with Large Data Sets
* Planning for Fall




## Deep Learning with Large Data Sets

There are times when you will need to be able to train a neural network with a dataset that is larger than can fit into the available memory of a machine.

So far, the first time I have encountered this issue is with an RNN that takes a sequence of input data.

For example, the project that I'm working with Radar and Satellite data has the following input structure:

```
year/month/day/
    radar/
        images (720x1832 float64) ~= 10MB (average ~150-300 scans per day)
    sat/
        band/
            images (each band scanned every 5 minutes)
    rain.csv
```

Assuming that the sat images are similar to the radar images, that gives us around 10GB of data per day.

The approach with Keras is to use a Data Generator to load the data incrementally when training the model.


## References:

[ImageDataGenerator](https://medium.com/datadriveninvestor/keras-imagedatagenerator-methods-an-easy-guide-550ecd3c0a92)  \[[source](https://github.com/keras-team/keras/blob/master/keras/preprocessing/image.py#L233)\]

[Multiprocess ImageDataGenerator](https://github.com/stratospark/keras-multiprocess-image-data-generator)

[Stanford Tutorial](https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly)

[Loading Large Data](https://medium.com/datadriveninvestor/keras-training-on-large-datasets-3e9d9dbc09d4)

[More Loading Large Datasets](https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/)

[Keras Feature extraction on large data](https://www.pyimagesearch.com/2019/05/27/keras-feature-extraction-on-large-datasets-with-deep-learning/)

[Keras Fit vs Fit_Generator](https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/)



## Keras Example

Here's an example from the Keras codebase that uses the ImageDataGenerator. You can think of this as a different approach that takes a given set of images and increases the data size by applying transformations to create new images based off of existing data.

In [0]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

from math import ceil

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


n_points = len(x_train)

steps_per_epoch = ceil(n_points / batch_size)


# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        steps_per_epoch=steps_per_epoch,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.8888 - acc: 0.3081 - val_loss: 1.5796 - val_acc: 0.4246
Epoch 2/100
1563/1563 [==============================] - 30s 19ms/step - loss: 1.5950 - acc: 0.4155 - val_loss: 1.4498 - val_acc: 0.4679
Epoch 3/100
1563/1563 [==============================] - 30s 19ms/step - loss: 1.4648 - acc: 0.4669 - val_loss: 1.3281 - val_acc: 0.5237
Epoch 4/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3778 - acc: 0.5040 - val_loss: 1.2068 - val_acc: 0.5666
Epoch 5/100
1563/1563 [==============================] - 30s 20ms/step - loss: 1.3005 - acc: 0.5367 - val_loss: 1.1448 - val_acc: 0.5890
Epoch 6/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.2436 - acc: 0.5574 - val_loss: 1.2455 - val_acc: 0.5662
Epoch 7/100
1563/1563 [==============================] - 31s 20ms/step